In [2]:
import pandas as pd
import numpy as np
import geopandas
import geopy
import requests
from IPython.display import IFrame
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from sklearn.cluster import KMeans
import shapely.geometry
import pyproj
import csv

In [157]:
#Strait of the real limit
Smin_lng = 2.225684
Smin_lat = 48.885438
Smax_lng = 2.253922
Smax_lat = 48.897599

#number of cubes in two dimension
lng_count = 20
lat_count = 13

#get each step of two dimension
delta_lng = Smax_lng - Smin_lng
delta_lat = Smax_lat - Smin_lat

step_lng = delta_lng/lng_count
step_lat = delta_lat/lat_count

min_lng = Smin_lng + step_lng/2
min_lat = Smin_lat + step_lat/2
max_lng = Smax_lng - step_lng/2
max_lat = Smax_lat - step_lat/2

print(min_lng,min_lat,max_lng,max_lat)

2.22638995 48.88590573076923 2.2532160500000002 48.89713126923077


In [158]:
gridpoints = []
x = min_lng
while x < max_lng:
    y = min_lat
    while y < max_lat:
        p = (x,y)
        gridpoints.append(p)
        y += step_lat
    x += step_lng  

In [159]:
with open('grid.csv', 'w') as of:
    of.write('longitude,latitude\n')
    for p in gridpoints:
        of.write('{}\n'.format(str(p)))

In [112]:
LD_grid = pd.read_csv("grid.csv",engine='python',sep=';')
LD_grid.insert(0, 'District', range(1, 1 + len(LD_grid)))
LD_grid.insert(0, 'Area(km2)', '1')
print(LD_grid)

  Area(km2)  District                                 longitude,latitude
0         1         1  rowrowrowrowrowrowrowrowrowrowrowrowrowrowrowr...


In [96]:
# create map, LD_tweets['text']
# set location that make Hong Kong at the centre of the web page
# set zoom_start = 12 for better visualization
map = folium.Map(tiles='cartodbpositron', location=[48.89114492, 2.240388888], zoom_start=14.6)

# add markers to the map to show the districts
markers_colors = []
for lat, lon, poi in zip(LD_grid['latitude'], LD_grid['longitude'],LD_grid['District']):
    label = folium.Popup(str(poi))
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        fill=True,
        fill_color='#ff6951',
        fill_opacity=0.4).add_to(map)
       
map.save('ld_grid.html')
display(IFrame('ld_grid.html', width = 1000, height = 900))

KeyError: 'latitude'

In [ ]:
# Define FourSquare Credential
CLIENT_ID = '3ZK4LUBW25RFO1X1FU5KNVRTUJQTZI1TR2GEH43OHI0FYTTQ' # your Foursquare ID
CLIENT_SECRET = 'DDRMTZ5UII0XSHBCFVVD5EPL3F1WJ4ZZM5VLZEEYG5CJJXYD' # your Foursquare Secret
VERSION = '20220128' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# Create a function to explore the Food venure from a given coordinates
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng, rad in zip(names, latitudes, longitudes, radius):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v3/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            rad, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LD_grid_venues = getNearbyVenues(names = LD_grid['District'],
                                   latitudes = LD_grid['latitude'],
                                   longitudes = LD_grid['longitude'],
                                   
                                   #assume district is a circle, determine radius by the equation of circle area: Area = pi * r^2
                                   radius = LD_grid['Area(km2)']
                                  )

In [ ]:
print(LD_grid_venues.groupby('District').count())

In [ ]:
https://api.foursquare.com/v2/venues/explore?&client_id={3ZK4LUBW25RFO1X1FU5KNVRTUJQTZI1TR2GEH43OHI0FYTTQ}&client_secret={DDRMTZ5UII0XSHBCFVVD5EPL3F1WJ4ZZM5VLZEEYG5CJJXYD}&v={20220202}&ll={2.252674},{48.887909}&radius={500}&limit={30}&categoryId=4d4b7105d754a06374d81259

In [7]:
def  count_list(std:list,tongji):
    from collections import Counter
    name=Counter()
    for  num in std:
        name[num]+=1
    print(name[tongji])
if __name__=='__main__':

IndentationError: expected an indented block (2422839095.py, line 7)

In [27]:
min_lng = 2.225684
min_lat = 48.885438
max_lng = 2.253922
max_lat = 48.897599
# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
to_proxy_transformer = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857')
to_original_transformer = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326')
# Create corners of rectangle to be transformed to a grid
sw = shapely.geometry.Point((min_lng, min_lat))
ne = shapely.geometry.Point((max_lng, max_lat))

stepsize = 135 # 5 km grid step size

# Project corners to target projection
transformed_sw = to_proxy_transformer.transform(sw.x, sw.y) # Transform NW point to 3857
transformed_ne = to_proxy_transformer.transform(ne.x, ne.y) # .. same for SE

# Iterate over 2D area
gridpoints = []
x = transformed_sw[0]
while x < transformed_ne[0]:
    y = transformed_sw[1]
    while y < transformed_ne[1]:
        p = shapely.geometry.Point(to_original_transformer.transform(x, y))
        gridpoints.append(p)
        y += stepsize
    x += stepsize  


In [28]:
len(gridpoints)

264